In [ ]:
NAME = "Chakrya Ros"

In [1]:
import $ivy.`org.scala-lang.modules::scala-parser-combinators:1.1.2`
import scala.util.parsing.combinator._
import $file.hw8stdlib
import hw8stdlib._

import $ivy.$                                                       

import scala.util.parsing.combinator._

import $file.$        

import hw8stdlib._

In [3]:
object ParserLettuce extends RegexParsers {

 //Convert number to nat
  def Int_to_nat(i:Int):Nat = i match {
    case 0 => Zero
    case x => Succ(Int_to_nat(x-1))
  }

  //Check if positive or negative
  def Int_to_Const(args: Int): Const = {
    if (args > 0)
      Const(Positive(Int_to_nat(args)))
    else
      Const(Negative(Int_to_nat(-args)))
  }

 //boolean Helper function
  def booleanHelp(b: Boolean) : Bool = b match {
    case true => True
    case false => False
}


//  p1 ~ p2 // sequencing: must match p1 followed by p2
//  p1 | p2 // alternation: must match either p1 or p2, with preference given to p1
//  p1.?    // optionality: may match p1 or not
//  p1.*   // repetition: matches any number of repetitions of p1
  def str: Parser[String] = """[a-zA-Z]+""".r
    
  def identifier: Parser[Expr] = """[a-zA-Z_]\w*""".r ^^ {x => Ident(x.toString) }
    
  def boolParse: Parser[Expr] = """True|False|true|false""".r ^^ {x => Bin(booleanHelp(x.toBoolean))}
    
  def PositiveParser: Parser[Expr] = """(0|[1-9]\d*)""".r ^^ { x  => Int_to_Const(x.toInt)}
    
  def NegativeParser: Parser[Expr] = "-"~"""(0|[1-9]\d*)""".r ^^ 
                                    { case (x ~ y)  => Const(Negative(Int_to_nat(y.toString.toInt)))}

  def symbol: Parser[String] = "+" | "-" | "*" | "^"
  
  def Logic_Parser : Parser[String] = """and|And|AND|or|Or|OR""".r
  
  def If_Parser : Parser[String] = """If|IF|if""".r
    
  def then_Parser : Parser[String] = """Then|then|THEN""".r
    
  def else_Parser : Parser[String] = """Else|else|ELSE""".r
    
  def let_Parser : Parser[String] = """Let|let|LET""".r
    
  def in_Parser : Parser[String] = """in|In|IN""".r
    
  def LetRec_Parser : Parser[String] = """LetRec |Let rec""".r
    
  def expr : Parser[Expr] = eval_Let | eval_LetRec | eval_FunDef | boolParse |
                            PositiveParser | NegativeParser | eval_Arth | 
                            eval_and_or | eval_equality|eval_Negate | 
                            eval_ifThenElse| identifier | eval_FunCall 
                            
    
    //function to calcuation operation 
  def eval_Arth = { "(" ~> expr ~ symbol ~ expr <~ ")"^^
    {
        
        case (e1 ~ "+" ~ e2) => Plus(e1, e2)
        case (e1 ~ "-" ~ e2) => Minus(e1, e2)
        case (e1 ~ "*" ~ e2) => Mult(e1, e2)
        case (e1 ~ "^" ~ e2) => Pow(e1, e2)

    }
  }
     //equality
  def eval_equality = { "(" ~> expr ~ "=" ~ expr <~ ")"^^
    {
        case (e1 ~ "=" ~ e2) => Eq(e1, e2)        
    }
                  
  }
    
  //Negate
  def eval_Negate = { "!" ~> expr^^
    {
        case (e) => Neg(e)       
    }
                  
  }
    
    //and and or
  def eval_and_or = { "(" ~> expr ~ Logic_Parser ~ expr <~ ")"^^
    {
        case e1 ~ "and" ~ e2 => And(e1, e2)
        case e1 ~ "And" ~ e2 => And(e1, e2) 
        case e1 ~ "AND" ~ e2 => And(e1, e2) 
        case e1 ~ "or" ~ e2 => Or(e1, e2)
        case e1 ~ "Or" ~ e2 => Or(e1, e2)
        case e1 ~ "OR" ~ e2 => Or(e1, e2)
    }
                  
  }
    
    

  //if x = 1 then 1 else 2
  def eval_ifThenElse = {If_Parser ~> expr ~ then_Parser ~ expr ~ else_Parser ~ expr^^
    {
        case(x ~ then ~ e1 ~ else_ ~ e2 ) => IfThenElse(x, e1, e2)

    }                         
 }
  //Let x = 1 in x+x 
  //Let x = functionbody in body of program
  def eval_Let = {let_Parser ~ str ~ "=" ~ expr ~ in_Parser ~ expr^^
    {
        case(let ~ x ~ equals ~ e1 ~ in ~ e2) => Let(x, e1, e2)
    }                         
 }
 
//  function(x) (x*x)
 def eval_FunDef = { """function""".r ~ "(" ~ str ~ ")" ~ expr ^^
    {
        case(f ~ op ~ id ~ cp ~ x) => FunDef(id, x)
    }                         
 }
 //FunCall
  //let fact = function(x) (x+x) in function<2>
 def eval_FunCall = {"(" ~ expr ~ "<" ~ expr ~ ">" ~ ")"^^
    {
        case(_ ~ fun ~ _ ~ e ~ _ ~ _) => FunCall(fun, e)
    }                         
 }

    
// LetRec fact = function(x) if x<=0 1 else x*fact(x-1) in fact(2)                     )
 def eval_LetRec = {LetRec_Parser ~ str ~ "=" ~ """function""".r ~ "(" ~ str ~ ")" ~ expr ~ in_Parser ~ expr^^
    {
        case(letRec ~ funName ~ equals ~ f ~ op ~ v ~ cp ~ e1 ~ in ~ body) => LetRec(funName, v, e1, body)
    }                   
 }
 
 
    
  def apply(input: String): Expr  = parseAll(expr, input) match {
    case Success(result, _) => result
    case failure : NoSuccess => scala.sys.error(failure.msg)
    
  }
    

}

      

def Check_Assert(x : Expr, expected : Value): Unit = {
    assert(eval(EmptyEnv, x) == expected)
    println("Pass Test!!")
}



//Ident
ParserLettuce("x")
ParserLettuce("Hello")

//Const
ParserLettuce("6")
ParserLettuce("-7")

//Bin
ParserLettuce("false")
ParserLettuce("true")

// Plus, Minus, Mult, Pow
ParserLettuce("(1+2)")
ParserLettuce("(3-4)")
ParserLettuce("(5*4)")
ParserLettuce("(4^2)")
ParserLettuce("((5+4)+2)")
ParserLettuce("((5-4)+2)")
ParserLettuce("((8+3)*1)")
ParserLettuce("((2*3)+(4-1))")

//Eq
ParserLettuce("(10=10)")
ParserLettuce("(4=5)")

////Neqate
ParserLettuce("!true")

// //And
ParserLettuce("(true and true)")
ParserLettuce("(2 And 4)")

// Or
ParserLettuce("(true or true)")
ParserLettuce("(8 or 1)")

//IfThenElse
ParserLettuce("if(x=1)then if(x=1)then (2+1) else 5 else 5")
ParserLettuce("if(x=1)then (x+1) else(3^2)")
ParserLettuce("if(1=2)then false else true")

//Let
ParserLettuce("Let x = 1 in (x+x)")
ParserLettuce("Let x = 1 in let y = 2 in ((y*x)+x)")

// //FunDef
ParserLettuce("function(x) (x*x)")

// //FunCall
ParserLettuce("(good<2>)")

ParserLettuce("Let square = function(x) (x*x) in (square<2>)")
ParserLettuce("Let g = function(x) (x+x) in (g<5>)")

//Let Rec
ParserLettuce("LetRec fact = function(x) if(x=0)then 1 else (x*(fact<(x-1)>)) in (fact<2>)")


defined object ParserLettuce
defined function Check_Assert
res2_2: Expr = Ident("x")
res2_3: Expr = Ident("Hello")
res2_4: Expr = Const(Positive(Succ(Succ(Succ(Succ(Succ(Succ(Zero))))))))
res2_5: Expr = Const(Negative(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Zero)))))))))
res2_6: Expr = Bin(False)
res2_7: Expr = Bin(True)
res2_8: Expr = Plus(
  Const(Positive(Succ(Zero))),
  Const(Positive(Succ(Succ(Zero))))
)
res2_9: Expr = Minus(
  Const(Positive(Succ(Succ(Succ(Zero))))),
  Const(Positive(Succ(Succ(Succ(Succ(Zero))))))
)
res2_10: Expr = Mult(
  Const(Positive(Succ(Succ(Succ(Succ(Succ(Zero))))))),
  Const(Positive(Succ(Succ(Succ(Succ(Zero))))))
)
res2_11: Expr = Pow(
  Const(Positive(Succ(Succ(Succ(Succ(Zero)))))),
  Const(Positive(Succ(Succ(Zero))))
)
res2_12: Expr = Plus(
  Plus(
    Const(Positive(Succ(Succ(Succ(Succ(Succ(Zero))))))),
    Const(Positive(Succ(Succ(Succ(Succ(Zero))))))
  ),
  Const(Positive(Succ(Succ(Zero))))
)
res2_13: Expr = Plus(
  Minus(
    Const(Positive(Succ(Succ(Su

In [4]:
Check_Assert(ParserLettuce("((5+4)-2)"), NumVal(Positive(seven)))
Check_Assert(ParserLettuce("((2*3)+(4-1))"), NumVal(Positive(nine)))
Check_Assert(ParserLettuce("(2^2)"), NumVal(Positive(four)))
Check_Assert(ParserLettuce("(4=5)"), BinVal(False))
Check_Assert(ParserLettuce("(6=6)"), BinVal(True))
Check_Assert(ParserLettuce("!(6=6)"), BinVal(False))
Check_Assert(ParserLettuce("!true"), BinVal(False))
Check_Assert(ParserLettuce("!false"), BinVal(True))
Check_Assert(ParserLettuce("!!false"), BinVal(False))
Check_Assert(ParserLettuce("!(6=6)"), BinVal(False))
Check_Assert(ParserLettuce("(false=false)"), BinVal(True))
Check_Assert(ParserLettuce("(true=true)"), BinVal(True))
Check_Assert(ParserLettuce("(true=true)"), BinVal(True))
Check_Assert(ParserLettuce("(true and true)"), BinVal(True))
Check_Assert(ParserLettuce("(true or false)"), BinVal(True))
Check_Assert(ParserLettuce("if(1=1)then (1+1) else 3"), NumVal(Positive(two)))
Check_Assert(ParserLettuce("if(1=2)then false else true"), BinVal(True))
Check_Assert(ParserLettuce("if(5=5)then (2-3) else (5-4)"), NumVal(Negative(one)))
Check_Assert(ParserLettuce("Let x = 2 in (x+x)"), NumVal(Positive(four)))
Check_Assert(ParserLettuce("Let square = function(x) (x*x) in (square<2>)"), NumVal(Positive(four))) 
Check_Assert(ParserLettuce("LetRec fact = function(x) If(x=0)then 1 Else (x*(fact<(x-1)>)) in (fact<3>)"), NumVal(Positive(six)))


Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!
Pass Test!!


In [7]:

def readInput {
    
    var bool_ = true
    var scanner = new java.util.Scanner(System.in)
    println("Welcome to Scala!!!!")
    println("Please enter your input or (quit) to quit")
    println()
    while (bool_ == true) {
        var userInput = scanner.nextLine()
        while(userInput =="")
        {
            println("Your input was empty. Please enter your input again.")
            println()
            userInput = scanner.nextLine()
            if(userInput == "quit")
            {
                bool_ = false
            }
        }
        if(userInput == "quit")
        {
            bool_ = false
        }
        else {
            var parsed = ParserLettuce(userInput)
            var eval_ = eval(EmptyEnv, parsed)
            println("Your input is " + userInput)
            println()
            println("Parsed to: " + parsed)
            println()
            println("Evaluated > " + eval_)
           println()
            
        }
    }
}


defined function readInput

In [ ]:
readInput

Welcome to Scala!!!!
Please enter your input or (quit) to quit

5
Your input is 5

Parsed to: Const(Positive(Succ(Succ(Succ(Succ(Succ(Zero)))))))

Evaluated > NumVal(Positive(Succ(Succ(Succ(Succ(Succ(Zero)))))))



In [ ]:
//Let x = 1 in let y = 2 in (x*y)
// Let square = function(x) (x*x) in (square<2>)
// LetRec fact = function(x) If(x=0)then 1 Else (x*(fact<(x-1)>)) in (fact<3>)
// if(1=1)then (1+1) else 3